In [1]:
# Beat tracking example
import librosa

# 1. Get the file path to an included audio example
filename = librosa.example('nutcracker')


# 2. Load the audio as a waveform `y`
#    Store the sampling rate as `sr`
y, sr = librosa.load(filename)

# 3. Run the default beat tracker
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

# 4. Convert the frame indices of beat events into timestamps
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

Estimated tempo: 107.67 beats per minute


In [ ]:
# Feature extraction example
import numpy as np
import librosa

# Load the example clip
y, sr = librosa.load(librosa.ex('nutcracker'))

# Set the hop length; at 22050 Hz, 512 samples ~= 23ms
hop_length = 512

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

# Beat track on the percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr)

# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

# And the first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)

# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)

# Compute chroma features from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames
beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)

# Finally, stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [2]:
import librosa
import librosa.display
import numpy as np

# Load the audio file
audio_file = 'data/Vibe - Taeyang.mp3'
y, sr = librosa.load(audio_file, sr=22050)

# Compute the Mel spectrogram
S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
log_S = librosa.power_to_db(S, ref=np.max)

# Compute the MFCC features
mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)

# Convert the MFCC features into a sequence of discrete notes
notes = []
for i in range(mfcc.shape[1]):
    note = np.argmax(mfcc[:, i])
    notes.append(note)

# Normalize the notes to a range of 0-1
notes = np.array(notes)
notes = (notes - np.min(notes)) / (np.max(notes) - np.min(notes))

# Modify the notes to create a new genre
# ...

# Convert the modified notes back into MFCC features
notes = (notes * (np.max(notes) - np.min(notes))) + np.min(notes)
mfcc = np.zeros((13, len(notes)))
for i, note in enumerate(notes):
    mfcc[int(note), i] = 1

# Convert the MFCC features back into audio
S = librosa.feature.inverse.mfcc_to_mel(mfcc)
S = librosa.db_to_power(S)
y = librosa.feature.inverse.mel_to_audio(S, sr=sr)

# Save the audio file
librosa.output.write_wav('data/new_audio_file.wav', y, sr)


C:\Users\jaden\AppData\Local\Temp\ipykernel_22984\546142811.py:10: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)


AttributeError: module 'librosa' has no attribute 'output'

In [4]:
import librosa

# Load the audio file
audio_file = "data/Vibe - Taeyang.mp3"
y, sr = librosa.load(audio_file)

# Extract features from the audio
chroma = librosa.feature.chroma_stft(y=y, sr=sr)
mfcc = librosa.feature.mfcc(y=y, sr=sr)

# Convert the features to a numeric format
chroma_data = chroma.tolist()
mfcc_data = mfcc.tolist()

# Save the data to a file
with open("data/chroma_data.txt", "w") as f:
    for row in chroma_data:
        f.write(" ".join([str(x) for x in row]) + "\n")

with open("data/mfcc_data.txt", "w") as f:
    for row in mfcc_data:
        f.write(" ".join([str(x) for x in row]) + "\n")
